# Modeling and Machine Learning --- Fixed Effects and Clustered Standard Errors

汇报人：李子明

本章主要介绍在面板数据回归时用于处理遗漏变量的固定效应模型和用于处理残差依赖性问题的标准误聚类。

我们关注一个在公司金融领域非常经典的面板数据回归，将公司资本投资建模为公司现金流和公司成长机会的函数， 如Fazzari(1988)、Erickson&Whited(2012)、Gulen&Ion(2015)等。我们将在下面证明，基于年度数据的回归产生的结果与基于文献中季度数据的估计结果相似，都说明了投资与两个解释变量之间的正相关关系。

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import linearmodels as lm
from regtabletotext import prettify_result, prettify_results

我们从Compustat中获取公司资产负债表和损益表数据，从CRSP中获取市场估值数据。

In [2]:
tidy_finance = sqlite3.connect(database="tidy_finance_python.sqlite")

crsp_monthly = pd.read_sql_query(
  sql="SELECT gvkey, month, mktcap FROM crsp_monthly",
  con=tidy_finance,
  parse_dates={"month"}
)

compustat = pd.read_sql_query(
  sql="SELECT datadate, gvkey, year, at, be, capx, oancf, txdb FROM compustat",
  con=tidy_finance,
  parse_dates={"datadate"}
)

构建变量：资本投资、现金流和托宾q。其中投资和现金流通过公司滞后的总资产来标准化，使其与公司规模相对应。而托宾q是资本市场价值与其重置成本的比率，我们遵循了Gulen和Ion(2015)的构建方法。最后，我们只保留所有感兴趣变量均不缺失且报告的资产账面价值严格为正的观测值。

In [3]:
data_investment = (compustat
  .assign(month = lambda x: (pd.to_datetime(x["datadate"]).dt.to_period('M').dt.to_timestamp()))
  .merge(compustat.get(["gvkey", "year", "at"])
          .rename(columns={"at": "at_lag"})
          .assign(year = lambda x: x["year"]+1), 
         on=["gvkey", "year"], how="left") # 将“at”列滞后一期
  .query("at > 0 and at_lag > 0")
  .assign(investment = lambda x: x["capx"]/x["at_lag"],
          cash_flows = lambda x: x["oancf"]/x["at_lag"])                   
) # 资本支出和经营活动净现金流除以上一期总资产，标准化投资和现金流

data_investment = (data_investment
  .merge(data_investment.get(["gvkey", "year", "investment"])
          .rename(columns={"investment": "investment_lead"})
          .assign(year = lambda x: x["year"]-1), 
         on=["gvkey", "year"], how="left")
) # 将资本投资提前一期相当于将现金流和托宾q滞后一期

data_investment = (data_investment
  .merge(crsp_monthly, on=["gvkey", "month"], how="left")
  .assign(tobins_q = lambda x: ((x["mktcap"]+x["at"]-x["be"]+x["txdb"])/x["at"]))
  # 托宾q=(权益的市场价值+总资产的账面价值-权益的账面价值+递延所得税)/总资产的账面价值
  .get(["gvkey", "year", "investment_lead", "cash_flows", "tobins_q"])
  .dropna()
)

由于变量构造通常会导致与数据问题（例如报告错误）相关的极值，因此许多论文都包含变量的缩尾化处理。缩尾是将数据中的极端值替换为分位数边界值来减轻极端值对统计分析的影响。下面的winsorize函数实现了应用于分布两端的任何百分比的缩尾。随后，我们应用函数在1%的水平上对主要变量进行缩尾处理。

In [4]:
def winsorize(x, cut):
    tmp_x = x.copy()
    upper_quantile = np.nanquantile(tmp_x, 1-cut)
    lower_quantile = np.nanquantile(tmp_x, cut)
    tmp_x[tmp_x > upper_quantile] = upper_quantile
    tmp_x[tmp_x < lower_quantile] = lower_quantile
    return tmp_x

data_investment = (data_investment
  .assign(investment_lead = lambda x: winsorize(x["investment_lead"], 0.01),
          cash_flows = lambda x: winsorize(x["cash_flows"], 0.01),
          tobins_q = lambda x: winsorize(x["tobins_q"], 0.01)
  )
)

主要变量的描述性统计如下。

In [5]:
data_investment_summary = data_investment[["investment_lead", "cash_flows", "tobins_q"]].describe(percentiles=[0.05, 0.5, 0.95])
data_investment_summary

,investment_lead,cash_flows,tobins_q
count,127447.000000,127447.000000,127447.000000
mean,0.057687,0.011102,1.996312
std,0.077181,0.274460,1.697512
min,0.000000,-1.563307,0.572763
5%,0.000675,-0.471674,0.793921
50%,0.032740,0.064063,1.388897
95%,0.206182,0.272274,5.365224
max,0.463684,0.479314,10.902323


# 固定效应模型

我们首先从基本回归模型开始。   
$Investment_{i,t}=\alpha+\beta_1Cashflows_{i,t-1}+\beta_2Tobin'sq_{i,t-1}+\epsilon_{i,t}$   
这里的$\epsilon_t$假设是独立同分布的，并且在时间和公司个体之间均呈正态分布。

In [6]:
model_ols = (lm.PanelOLS.from_formula(
    formula="investment_lead ~ cash_flows + tobins_q + 1", # 这里“+1”表示包含常数项
    data=data_investment.set_index(["gvkey", "year"]),
  ).fit()
)
prettify_result(model_ols) # prettify_result提供类似于R语言的输出

Panel OLS Model:
investment_lead ~ cash_flows + tobins_q + 1

Covariance Type: Unadjusted

Coefficients:
            Estimate  Std. Error  t-Statistic  p-Value
Intercept      0.042       0.000      127.470      0.0
cash_flows     0.049       0.001       61.814      0.0
tobins_q       0.007       0.000       57.045      0.0

Summary statistics:
- Number of observations: 127,447
- R-squared (incl. FE): 0.043, Within R-squared: 0.039
- F-statistic: 2,865.058, p-value: 0.000



正如预期的那样，回归结果显示两个变量的回归系数均显著为正。公司更高的现金流和更好的成长机会与公司更高的资本投资相关。然而简单的模型实际上可能将需要控制的变量遗漏，我们上述的基本模型中有很多无法解释的变化（由相当低的调整$R^2$表示），因此我们的系数偏差可能很严重。   

解决遗漏变量偏差问题的一种方法是通过包含固定效应，针对特定群体固定模型中的参数，来尽可能多地消除由群体无法观测差异造成的影响。为了控制不同公司间的个体差异，我们通过在基本模型中加入公司固定效应$\mu_i$，来捕捉所有年份特定公司的平均投资，该投资可能受到未观察到的公司特定变量的影响。  
$Investment_{i,t}=\mu_i+\beta_1Cashflows_{i,t-1}+\beta_2Tobin'sq_{i,t-1}+\epsilon_{i,t}$

In [7]:
model_fe_firm = (lm.PanelOLS.from_formula(
    formula=("investment_lead ~ cash_flows + tobins_q + EntityEffects"), # 在linearmodels库中，个体固定效应可以通过EntityEffects直接引入
    data=data_investment.set_index(["gvkey", "year"]),
  ).fit()
)
prettify_result(model_fe_firm)

Panel OLS Model:
investment_lead ~ cash_flows + tobins_q + EntityEffects

Covariance Type: Unadjusted

Coefficients:
            Estimate  Std. Error  t-Statistic  p-Value
cash_flows     0.014       0.001       15.268      0.0
tobins_q       0.011       0.000       82.006      0.0

Included Fixed Effects:
        Total
Entity  14349

Summary statistics:
- Number of observations: 127,447
- R-squared (incl. FE): 0.585, Within R-squared: 0.057
- F-statistic: 3,433.175, p-value: 0.000



回归结果显示公司层面存在大量无法解释的差异，在加入公司固定效应后，整体调整$R^2$上升至50%以上。回归结果中的组内$R^2$表示每个公司现金流和托宾q对平均投资的解释力，解释变量的系数的大小略有变化，但符号没有变化。

宏观经济等系统性因素会对所有公司造成影响，使得所有公司的投资水平随着时间变化。为了控制随时间变化的不可观测变量的影响，我们通过在上述模型中加入时间固定效应$\lambda_t$，构建双向固定效应模型。  
$Investment_{i,t}=\mu_i+\lambda_t+\beta_1Cashflows_{i,t-1}+\beta_2Tobin'sq_{i,t-1}+\epsilon_{i,t}$

In [8]:
model_fe_firmyear = (lm.PanelOLS.from_formula(
    formula=("investment_lead ~ cash_flows + tobins_q + EntityEffects + TimeEffects"), # 同样，时间固定效应可以通过TimeEffects直接引入
    data=data_investment.set_index(["gvkey", "year"]),
  ).fit()
)
prettify_result(model_fe_firmyear)

Panel OLS Model:
investment_lead ~ cash_flows + tobins_q + EntityEffects 
 + TimeEffects

Covariance Type: Unadjusted

Coefficients:
            Estimate  Std. Error  t-Statistic  p-Value
cash_flows     0.018       0.001       19.409      0.0
tobins_q       0.010       0.000       75.332      0.0

Included Fixed Effects:
        Total
Entity  14349
Time       35

Summary statistics:
- Number of observations: 127,447
- R-squared (incl. FE): 0.607, Within R-squared: 0.057
- F-statistic: 2,979.799, p-value: 0.000



回归结果显示，加入时间固定效应对调整$R^2$和系数产生的影响都很小，这表明系数不是由随时间变化的遗漏变量驱动的。

In [9]:
prettify_results([model_ols, model_fe_firm, model_fe_firmyear])

Outcome        investment_lead  investment_lead  investment_lead

Intercept      0.042 (127.47)
cash_flows      0.049 (61.81)    0.014 (15.27)    0.018 (19.41)
tobins_q        0.007 (57.04)    0.011 (82.01)    0.01 (75.33)

Fixed effects                       Entity        Entity, Time
VCOV type        Unadjusted       Unadjusted       Unadjusted
Observations       127,447          127,447          127,447
R2 (incl. FE)       0.043            0.585            0.607
Within R2           0.039            0.057            0.057


# 标准误聚类

除了有偏估计之外，我们通常还必须处理残差之间潜在的复杂依赖性。残差的这种依赖性使高斯马尔科夫假定中的球形扰动项条件被破坏，并影响标准误的有效性，从而无法可靠地解释统计显著性。在前文设置中，残差可能在给定公司的各个年份之间相关（时间序列依赖性），或者残差可能给在给定时期的不同公司之间相关（横截面依赖性）。处理这种依赖性的常见方法是使用聚类标准误。该方法认为聚类内残差的相关性可以是任何形式的，但随着聚类数量的增加，聚类稳健标准误会变得具有一致性。因此如果我们构建足够多的聚类（通常至少30到50个聚类可以视作是足够的），聚类稳健标准误就可以作为真实标准误的一致估计。

下面我们放弃残差的独立同分布假设，认为残差具有横截面依赖性，在公司层面对标准误进行单向聚类。

In [10]:
model_cluster_firm = lm.PanelOLS.from_formula(
  formula=("investment_lead ~ cash_flows + tobins_q + EntityEffects + TimeEffects"),
  data=data_investment.set_index(["gvkey", "year"]),
).fit(cov_type="clustered", 
      cluster_entity=True, 
      cluster_time=False)
prettify_result(model_cluster_firm)

Panel OLS Model:
investment_lead ~ cash_flows + tobins_q + EntityEffects 
 + TimeEffects

Covariance Type: Clustered

Coefficients:
            Estimate  Std. Error  t-Statistic  p-Value
cash_flows     0.018       0.002       10.603      0.0
tobins_q       0.010       0.000       33.374      0.0

Included Fixed Effects:
        Total
Entity  14349
Time       35

Summary statistics:
- Number of observations: 127,447
- R-squared (incl. FE): 0.607, Within R-squared: 0.057
- F-statistic: 2,979.799, p-value: 0.000



随后，我们认为残差同时具有横截面依赖性和时间序列依赖性，在公司和时间维度进行双向聚类。

In [11]:
model_cluster_firmyear = lm.PanelOLS.from_formula(
  formula=("investment_lead ~ cash_flows + tobins_q + EntityEffects + TimeEffects"),
  data=data_investment.set_index(["gvkey", "year"]),
).fit(cov_type="clustered", 
      cluster_entity=True, 
      cluster_time=True)
prettify_result(model_cluster_firmyear)

Panel OLS Model:
investment_lead ~ cash_flows + tobins_q + EntityEffects 
 + TimeEffects

Covariance Type: Clustered

Coefficients:
            Estimate  Std. Error  t-Statistic  p-Value
cash_flows     0.018       0.002        9.142      0.0
tobins_q       0.010       0.001       14.677      0.0

Included Fixed Effects:
        Total
Entity  14349
Time       35

Summary statistics:
- Number of observations: 127,447
- R-squared (incl. FE): 0.607, Within R-squared: 0.057
- F-statistic: 2,979.799, p-value: 0.000



In [12]:
prettify_results([model_fe_firmyear, model_cluster_firm, model_cluster_firmyear])

Outcome        investment_lead  investment_lead  investment_lead

cash_flows      0.018 (19.41)    0.018 (10.6)     0.018 (9.14)
tobins_q        0.01 (75.33)     0.01 (33.37)     0.01 (14.68)

Fixed effects   Entity, Time     Entity, Time     Entity, Time
VCOV type        Unadjusted        Clustered        Clustered
Observations       127,447          127,447          127,447
R2 (incl. FE)       0.607            0.607            0.607
Within R2           0.057            0.057            0.057


上表展示了在不同残差假设下的回归结果比较。在第一列独立同分布假设下，可以看到现金流和托宾q的回归系数都非常显著。而在第二列横截面依赖性下，我们利用到公司层面的聚类稳健标准误，两个解释变量的回归系数的统计量都下降了一半，表明不同公司的残差之间存在高度的相关性。在第三列同时具有横截面和时间序列依赖性下，我们利用到双向聚类的稳健标准误，两个解释变量的回归系数的统计量再次下降，尤其是托宾q再次下降了一半。然而即使放松了标准误的独立同分布假设，可以看到这两个系数仍然在1%的显著性水平上具有统计显著性，因此公司资本投资可以由公司现金流和公司成长机会这两个变量解释。

练习一：   
使用WRDS的季度Compustat数据估计具有双向聚类标准误差的双向固定效应模型。

练习二：   
按照Peters和Taylor(2017)的方法，将托宾q计算为已发行股本的市场价值mktcap加上债务账面价值(dltt+dlc)减去流动资产atc和一切除以财产、厂房和设备的账面价值ppegt。托宾q的度量之间有什么相关性？对双向固定效应回归有何影响？